In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets as datasets

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

#Split test data into validation and test:
x_val, x_test = x_test[:5000], x_test[5000:]
y_val, y_test = y_test[:5000], y_test[5000:]

#Get the shape of the training data
x_train.shape

(60000, 28, 28)

In [3]:
#Pre-process training data
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0

#Pre-process validation data
x_val = x_val.reshape(-1, 28, 28, 1).astype("float32") / 255.0

#Pre-process test data
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0



In [4]:
#Create a Convolutional Neural Network using functional API:
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Conv2D(32, 3, activation="relu")(inputs)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(64, 3, activation="relu")(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(128, 3, activation="relu")(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation="relu")(x)
outputs = keras.layers.Dense(10)(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")



Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-09-18 15:21:25.979539: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-18 15:21:25.979639: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
#Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

#Check if GPU available:
tf.test.is_gpu_available()

#Train model with GPU
with tf.device("/GPU:0"):
    model.fit(x_train, y_train, batch_size=64, epochs=3, validation_data=(x_val, y_val))


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-09-18 15:21:26.139099: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-18 15:21:26.139142: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-18 15:21:26.442142: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-18 15:21:26.442351: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-18 15:21:26.597756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/3
938/938 [==============================] - ETA: 0s - loss: 0.1679 - accuracy: 0.9482

2022-09-18 15:21:37.581136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 12s 11ms/step - loss: 0.1679 - accuracy: 0.9482 - val_loss: 0.0627 - val_accuracy: 0.9800
Epoch 2/3
938/938 [==============================] - 10s 10ms/step - loss: 0.0448 - accuracy: 0.9866 - val_loss: 0.0582 - val_accuracy: 0.9802
Epoch 3/3
938/938 [==============================] - 10s 10ms/step - loss: 0.0329 - accuracy: 0.9899 - val_loss: 0.0412 - val_accuracy: 0.9862


In [6]:

#Evaluate the model
history = model.evaluate(x_test, y_test, batch_size=64, verbose=2)

79/79 - 0s - loss: 0.0206 - accuracy: 0.9938


[0.020643480122089386, 0.9938000440597534]

In [ ]:
#Get history
history = model.history.history